In [ ]:
# Import necessary libraries
import glob
from pathlib import Path
import warnings

import lpips
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from skimage.color import lab2rgb, rgb2lab
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from tqdm.notebook import tqdm

import torch
import torchvision.models as models
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
from colormath.color_objects import LabColor, sRGBColor

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*negative Z values.*')

In [ ]:
# Device configuration
def get_device():
    if torch.cuda.is_available():
        dev = torch.device("cuda")
        name = torch.cuda.get_device_name(0)
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        dev = torch.device("mps")
        name = "Apple Silicon"
    else:
        dev = torch.device("cpu")
        name = "CPU"
    return dev, name

device, device_name = get_device()
print(f"Using device: {device_name} ({device})")

In [ ]:
# Configuration
MODEL_ID = 'C'
BATCH_SIZE = 64
IMG_SIZE = 256
EPOCHS = 50
SAVE_INTERVAL = 10

# Model-specific configuration
USE_GAN = True
USE_L1 = True
USE_PERCEPTUAL = True
LAMBDA_L1 = 100.0
LAMBDA_PERCEPTUAL = 10.0

print(f"Model {MODEL_ID} Configuration:")
print(f"  GAN: {USE_GAN}")
print(f"  L1 Loss: {USE_L1} (λ₁ = {LAMBDA_L1})")
print(f"  Perceptual Loss: {USE_PERCEPTUAL} (λ₂ = {LAMBDA_PERCEPTUAL})")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")

In [ ]:
# Dataset class
class ColorizeDataset(Dataset):
    def __init__(self, root_dir, img_size=256, split='train'):
        self.root_dir = Path(root_dir)
        self.img_size = img_size
        self.split = split
        
        color_dir = self.root_dir / f"{split}_color"
        self.color_paths = sorted(glob.glob(str(color_dir / "*.jpg")))
        
        black_dir = self.root_dir / f"{split}_black"
        self.black_paths = sorted(glob.glob(str(black_dir / "*.jpg")))
        
        assert len(self.color_paths) == len(self.black_paths)
        
        # Data augmentation for training set
        if split == 'train':
            self.transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(degrees=15),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                transforms.ToTensor()
            ])
        else:
            # No augmentation for validation and test sets
            self.transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor()
            ])
        
        print(f"Loaded {len(self.color_paths)} {split} image pairs")
    
    def __len__(self):
        return len(self.color_paths)
    
    def __getitem__(self, idx):
        color_img = Image.open(self.color_paths[idx]).convert('RGB')
        color_img = self.transform(color_img)
        
        gray_img = Image.open(self.black_paths[idx]).convert('L')
        gray_img = self.transform(gray_img)
        
        color_img_np = (color_img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        lab_img = rgb2lab(color_img_np)
        
        L = lab_img[:, :, 0] / 50.0 - 1.0
        ab = lab_img[:, :, 1:] / 128.0
        
        L = torch.from_numpy(L).unsqueeze(0).float()
        ab = torch.from_numpy(ab).permute(2, 0, 1).float()
        
        return L, ab, gray_img

# Load datasets
train_dataset = ColorizeDataset('../data/colorize_dataset/data', img_size=IMG_SIZE, split='train')
val_dataset = ColorizeDataset('../data/colorize_dataset/data', img_size=IMG_SIZE, split='val')
test_dataset = ColorizeDataset('../data/colorize_dataset/data', img_size=IMG_SIZE, split='test')

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True if torch.cuda.is_available() else False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True if torch.cuda.is_available() else False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True if torch.cuda.is_available() else False)

# Display dataset statistics
print(f"\n{'='*60}")
print(f"Dataset Statistics:")
print(f"{'='*60}")
print(f"Training set:   {len(train_dataset):4d} images ({len(train_loader):3d} batches)")
print(f"Validation set: {len(val_dataset):4d} images ({len(val_loader):3d} batches)")
print(f"Test set:       {len(test_dataset):4d} images ({len(test_loader):3d} batches)")
print(f"Image size:     {IMG_SIZE}×{IMG_SIZE} pixels")
print(f"Batch size:     {BATCH_SIZE}")
print(f"{'='*60}\n")

## U-Net Generator Architecture

In [ ]:
# U-Net components
class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_dropout=False):
        super().__init__()
        if down:
            self.conv = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2, inplace=True)
            )
        else:
            self.conv = nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        
        self.use_dropout = use_dropout
        if use_dropout:
            self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.conv(x)
        if self.use_dropout:
            x = self.dropout(x)
        return x

class UNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=2):
        super().__init__()
        
        self.down1 = nn.Conv2d(in_channels, 64, 4, 2, 1)
        self.down2 = UNetBlock(64, 128, down=True)
        self.down3 = UNetBlock(128, 256, down=True)
        self.down4 = UNetBlock(256, 512, down=True)
        self.down5 = UNetBlock(512, 512, down=True)
        self.down6 = UNetBlock(512, 512, down=True)
        self.down7 = UNetBlock(512, 512, down=True)
        
        self.bottleneck = nn.Sequential(
            nn.Conv2d(512, 512, 4, 2, 1),
            nn.ReLU(inplace=True)
        )
        
        self.up1 = UNetBlock(512, 512, down=False, use_dropout=True)
        self.up2 = UNetBlock(1024, 512, down=False, use_dropout=True)
        self.up3 = UNetBlock(1024, 512, down=False, use_dropout=True)
        self.up4 = UNetBlock(1024, 512, down=False)
        self.up5 = UNetBlock(1024, 256, down=False)
        self.up6 = UNetBlock(512, 128, down=False)
        self.up7 = UNetBlock(256, 64, down=False)
        
        self.final = nn.Sequential(
            nn.ConvTranspose2d(128, out_channels, 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        
        bottleneck = self.bottleneck(d7)
        
        u1 = self.up1(bottleneck)
        u2 = self.up2(torch.cat([u1, d7], dim=1))
        u3 = self.up3(torch.cat([u2, d6], dim=1))
        u4 = self.up4(torch.cat([u3, d5], dim=1))
        u5 = self.up5(torch.cat([u4, d4], dim=1))
        u6 = self.up6(torch.cat([u5, d3], dim=1))
        u7 = self.up7(torch.cat([u6, d2], dim=1))
        
        output = self.final(torch.cat([u7, d1], dim=1))
        return output

# Initialize generator
generator = UNetGenerator(in_channels=1, out_channels=2).to(device)
print("U-Net Generator initialized")

## PatchGAN Discriminator

In [ ]:
# PatchGAN Discriminator
class PatchGANDiscriminator(nn.Module):
    """70x70 PatchGAN discriminator for conditional GAN"""
    def __init__(self, in_channels=3):
        super().__init__()
        
        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, 2, 1)]
            if normalization:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        self.model = nn.Sequential(
            *discriminator_block(in_channels, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.Conv2d(512, 1, 4, 1, 1)
        )
    
    def forward(self, L, ab):
        x = torch.cat([L, ab], dim=1)
        return self.model(x)

# Initialize discriminator
discriminator = PatchGANDiscriminator(in_channels=3).to(device)
print("PatchGAN Discriminator initialized")

## Loss Functions

In [ ]:
# Perceptual Loss using VGG16
class PerceptualLoss(nn.Module):
    """Perceptual loss using VGG16 features"""
    def __init__(self):
        super().__init__()
        try:
            from torchvision.models import VGG16_Weights
            vgg = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1).features
        except ImportError:
            vgg = models.vgg16(pretrained=True).features
        
        self.feature_extractor = nn.Sequential(*list(vgg.children())[:16]).eval()
        
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
        
        self.criterion = nn.L1Loss()
    
    def forward(self, generated, target):
        gen_rgb = torch.cat([generated, generated[:, :1, :, :]], dim=1)
        target_rgb = torch.cat([target, target[:, :1, :, :]], dim=1)
        
        mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(generated.device)
        std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(generated.device)
        
        gen_normalized = (gen_rgb - mean) / std
        target_normalized = (target_rgb - mean) / std
        
        gen_features = self.feature_extractor(gen_normalized)
        target_features = self.feature_extractor(target_normalized)
        
        return self.criterion(gen_features, target_features)

# GAN Loss
class GANLoss(nn.Module):
    """GAN loss (BCE with logits)"""
    def __init__(self):
        super().__init__()
        self.criterion = nn.BCEWithLogitsLoss()
    
    def forward(self, prediction, target_is_real):
        if target_is_real:
            target = torch.ones_like(prediction)
        else:
            target = torch.zeros_like(prediction)
        return self.criterion(prediction, target)

# Initialize loss functions
l1_loss = nn.L1Loss()
perceptual_loss = PerceptualLoss().to(device)
gan_loss = GANLoss()

print("All loss functions initialized")

## Training Setup

In [ ]:
# Optimizers
gen_optimizer = optim.Adam(generator.parameters(), lr=2e-4, betas=(0.5, 0.999))
disc_optimizer = optim.Adam(discriminator.parameters(), lr=2e-4, betas=(0.5, 0.999))

# Training history
history = {
    'epoch': [],
    'train_loss': [], 
    'val_loss': [],
    'gen_loss': [],
    'disc_loss': []
}

# Create save directories
save_dir = Path(f'../models/{MODEL_ID}')
results_dir = Path('../results')
save_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

print(f"Model will be saved to: {save_dir}")
print(f"Results will be saved to: {results_dir}")

## Training Loop

In [ ]:
# Validation function
def validate_epoch(generator, val_loader, device):
    generator.eval()
    epoch_loss = 0.0
    
    with torch.no_grad():
        for L, ab_real, _ in tqdm(val_loader, desc="Validating", leave=False):
            L = L.to(device)
            ab_real = ab_real.to(device)
            
            ab_fake = generator(L)
            
            # Calculate loss: L1 + Perceptual (skip GAN loss for validation)
            loss = l1_loss(ab_fake, ab_real) * LAMBDA_L1 + perceptual_loss(ab_fake, ab_real) * LAMBDA_PERCEPTUAL
            
            epoch_loss += loss.item()
    
    return epoch_loss / len(val_loader)

# Visualization function for validation samples
def visualize_validation_samples(generator, val_loader, device, epoch, save_dir, num_samples=5):
    generator.eval()
    
    with torch.no_grad():
        L, ab_real, gray_img = next(iter(val_loader))
        L = L[:num_samples].to(device)
        ab_real = ab_real[:num_samples]
        gray_img = gray_img[:num_samples]
        
        ab_fake = generator(L).cpu()
        L = L.cpu()
        
        fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4 * num_samples))
        
        for i in range(num_samples):
            # Grayscale input
            axes[i, 0].imshow(gray_img[i].squeeze(), cmap='gray')
            axes[i, 0].set_title('Input (Grayscale)', fontweight='bold')
            axes[i, 0].axis('off')
            
            # Generated colorization
            L_np = ((L[i].squeeze().numpy() + 1.0) * 50.0)
            ab_fake_np = ab_fake[i].permute(1, 2, 0).numpy() * 128.0
            lab_fake = np.zeros((IMG_SIZE, IMG_SIZE, 3))
            lab_fake[:, :, 0] = L_np
            lab_fake[:, :, 1:] = ab_fake_np
            rgb_fake = lab2rgb(lab_fake)
            
            axes[i, 1].imshow(rgb_fake)
            axes[i, 1].set_title('Generated Color', fontweight='bold')
            axes[i, 1].axis('off')
            
            # Ground truth
            ab_real_np = ab_real[i].permute(1, 2, 0).numpy() * 128.0
            lab_real = np.zeros((IMG_SIZE, IMG_SIZE, 3))
            lab_real[:, :, 0] = L_np
            lab_real[:, :, 1:] = ab_real_np
            rgb_real = lab2rgb(lab_real)
            
            axes[i, 2].imshow(rgb_real)
            axes[i, 2].set_title('Ground Truth', fontweight='bold')
            axes[i, 2].axis('off')
        
        plt.suptitle(f'Validation Samples - Epoch {epoch}', fontsize=16, fontweight='bold', y=1.0)
        plt.tight_layout()
        
        output_path = save_dir / f'val_samples_epoch_{epoch}.png'
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        print(f"  ✓ Validation samples saved to: {output_path.name}")
        
        # Display the plot in notebook
        plt.show()

In [ ]:
# Training function
def train_epoch(generator, discriminator, train_loader, gen_optimizer, disc_optimizer, epoch, device):
    generator.train()
    discriminator.train()
    
    epoch_gen_loss = 0.0
    epoch_disc_loss = 0.0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    for L, ab_real, _ in pbar:
        L = L.to(device)
        ab_real = ab_real.to(device)
        
        # ==================
        # Train Discriminator
        # ==================
        disc_optimizer.zero_grad()
        
        # Generate fake images
        ab_fake = generator(L)
        
        # Discriminator on real images
        pred_real = discriminator(L, ab_real)
        loss_real = gan_loss(pred_real, True)
        
        # Discriminator on fake images
        pred_fake = discriminator(L, ab_fake.detach())
        loss_fake = gan_loss(pred_fake, False)
        
        # Total discriminator loss
        loss_disc = (loss_real + loss_fake) * 0.5
        loss_disc.backward()
        disc_optimizer.step()
        
        # ================
        # Train Generator
        # ================
        gen_optimizer.zero_grad()
        
        # Generator wants discriminator to classify fake as real
        pred_fake = discriminator(L, ab_fake)
        loss_gan = gan_loss(pred_fake, True)
        
        # L1 loss
        loss_l1 = l1_loss(ab_fake, ab_real) * LAMBDA_L1
        
        # Perceptual loss
        loss_perceptual_val = perceptual_loss(ab_fake, ab_real) * LAMBDA_PERCEPTUAL
        
        # Total generator loss
        loss_gen = loss_gan + loss_l1 + loss_perceptual_val
        loss_gen.backward()
        gen_optimizer.step()
        
        epoch_gen_loss += loss_gen.item()
        epoch_disc_loss += loss_disc.item()
        
        pbar.set_postfix({
            'G_loss': f"{loss_gen.item():.4f}",
            'D_loss': f"{loss_disc.item():.4f}"
        })
    
    return epoch_gen_loss / len(train_loader), epoch_disc_loss / len(train_loader)

# Train model
best_val_loss = float('inf')

print(f"\n{'='*60}")
print(f"Starting Training - Model {MODEL_ID}")
print(f"{'='*60}\n")

for epoch in range(1, EPOCHS + 1):
    # Train
    avg_gen_loss, avg_disc_loss = train_epoch(
        generator, discriminator, train_loader, 
        gen_optimizer, disc_optimizer, epoch, device
    )
    
    # Validate
    avg_val_loss = validate_epoch(generator, val_loader, device)
    
    # Update history
    history['epoch'].append(epoch)
    history['gen_loss'].append(avg_gen_loss)
    history['disc_loss'].append(avg_disc_loss)
    history['train_loss'].append(avg_gen_loss)
    history['val_loss'].append(avg_val_loss)
    
    print(f"\nEpoch {epoch}/{EPOCHS}")
    print(f"  Generator Loss: {avg_gen_loss:.4f}")
    print(f"  Discriminator Loss: {avg_disc_loss:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f}")
    
    # Save best model based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        checkpoint = {
            'epoch': epoch,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'gen_optimizer_state_dict': gen_optimizer.state_dict(),
            'disc_optimizer_state_dict': disc_optimizer.state_dict(),
            'history': history,
            'best_val_loss': best_val_loss,
            'best_train_loss': avg_gen_loss
        }
        torch.save(checkpoint, save_dir / 'best_model.pt')
        print(f"  ✓ New best model saved! (Val Loss: {best_val_loss:.4f})")
    
    # Save checkpoint at intervals
    if epoch % SAVE_INTERVAL == 0:
        checkpoint_path = save_dir / f"checkpoint_epoch_{epoch}.pt"
        torch.save(checkpoint, checkpoint_path)
        print(f"  ✓ Checkpoint saved at epoch {epoch}")
        
        # Visualize validation samples every 10 epochs
        visualize_validation_samples(generator, val_loader, device, epoch, results_dir)
    
    # Save training history to CSV after each epoch
    history_df = pd.DataFrame(history)
    history_csv_path = results_dir / f'training_history_{MODEL_ID}.csv'
    history_df.to_csv(history_csv_path, index=False)
    
    # Clear cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print(f"\n{'='*60}")
print(f"Training completed!")
print(f"Best Validation Loss: {best_val_loss:.4f}")
print(f"Training history saved to: {history_csv_path}")
print(f"{'='*60}\n")

## Evaluation

In [ ]:
# Patch numpy for colormath compatibility
if not hasattr(np, 'asscalar'):
    np.asscalar = lambda x: x.item() if hasattr(x, 'item') else float(x)

# Evaluation metrics class
class ColorimetricEvaluator:
    def __init__(self, device):
        self.device = device
        self.lpips_model = lpips.LPIPS(net='alex').to(device)
        self.lpips_model.eval()
    
    def calculate_psnr(self, generated, target):
        gen_np = generated.cpu().numpy()
        target_np = target.cpu().numpy()
        psnr_values = []
        for i in range(gen_np.shape[0]):
            psnr = peak_signal_noise_ratio(target_np[i], gen_np[i], data_range=2.0)
            psnr_values.append(psnr)
        return np.mean(psnr_values)
    
    def calculate_ssim(self, generated, target):
        gen_np = generated.cpu().numpy()
        target_np = target.cpu().numpy()
        ssim_values = []
        for i in range(gen_np.shape[0]):
            gen_img = np.transpose(gen_np[i], (1, 2, 0))
            target_img = np.transpose(target_np[i], (1, 2, 0))
            ssim = structural_similarity(target_img, gen_img, data_range=2.0, channel_axis=2)
            ssim_values.append(ssim)
        return np.mean(ssim_values)
    
    def calculate_ciede2000(self, L, ab_generated, ab_target):
        L_np = ((L.cpu().numpy() + 1.0) * 50.0)
        ab_gen_np = ab_generated.cpu().numpy() * 128.0
        ab_target_np = ab_target.cpu().numpy() * 128.0
        
        delta_e_values = []
        for b in range(L_np.shape[0]):
            L_channel = L_np[b, 0]
            step = 8
            sample_indices = [(i, j) for i in range(0, L_channel.shape[0], step) for j in range(0, L_channel.shape[1], step)]
            
            pixel_deltas = []
            for i, j in sample_indices:
                lab_gen = LabColor(L_channel[i, j], ab_gen_np[b, 0, i, j], ab_gen_np[b, 1, i, j])
                lab_target = LabColor(L_channel[i, j], ab_target_np[b, 0, i, j], ab_target_np[b, 1, i, j])
                delta_e = delta_e_cie2000(lab_gen, lab_target)
                pixel_deltas.append(delta_e)
            
            delta_e_values.append(np.mean(pixel_deltas))
        return np.mean(delta_e_values)
    
    def calculate_lpips(self, L, ab_generated, ab_target):
        with torch.no_grad():
            batch_size = L.size(0)
            rgb_gen_list = []
            rgb_target_list = []
            
            for i in range(batch_size):
                L_np = ((L[i].cpu().squeeze().numpy() + 1.0) * 50.0)
                ab_gen_np = ab_generated[i].cpu().permute(1, 2, 0).numpy() * 128.0
                lab_gen = np.zeros((L_np.shape[0], L_np.shape[1], 3))
                lab_gen[:, :, 0] = L_np
                lab_gen[:, :, 1:] = ab_gen_np
                rgb_gen = lab2rgb(lab_gen)
                
                ab_target_np = ab_target[i].cpu().permute(1, 2, 0).numpy() * 128.0
                lab_target = np.zeros((L_np.shape[0], L_np.shape[1], 3))
                lab_target[:, :, 0] = L_np
                lab_target[:, :, 1:] = ab_target_np
                rgb_target = lab2rgb(lab_target)
                
                rgb_gen_list.append(rgb_gen)
                rgb_target_list.append(rgb_target)
            
            rgb_gen_tensor = torch.from_numpy(np.array(rgb_gen_list)).permute(0, 3, 1, 2).float() * 2 - 1
            rgb_target_tensor = torch.from_numpy(np.array(rgb_target_list)).permute(0, 3, 1, 2).float() * 2 - 1
            
            rgb_gen_tensor = rgb_gen_tensor.to(self.device)
            rgb_target_tensor = rgb_target_tensor.to(self.device)
            
            lpips_value = self.lpips_model(rgb_gen_tensor, rgb_target_tensor)
            return lpips_value.mean().item()
    
    def evaluate_batch(self, L, ab_generated, ab_target):
        metrics = {
            'PSNR': self.calculate_psnr(ab_generated, ab_target),
            'SSIM': self.calculate_ssim(ab_generated, ab_target),
            'CIEDE2000': self.calculate_ciede2000(L, ab_generated, ab_target),
            'LPIPS': self.calculate_lpips(L, ab_generated, ab_target)
        }
        return metrics

evaluator = ColorimetricEvaluator(device)
print("Evaluator initialized")

In [ ]:
# Evaluate on test set
generator.eval()

metrics_sum = {'PSNR': 0.0, 'SSIM': 0.0, 'CIEDE2000': 0.0, 'LPIPS': 0.0}
num_evaluated = 0

print(f"Evaluating Model {MODEL_ID} on test set...")
with torch.no_grad():
    for L, ab_real, _ in tqdm(test_loader, desc="Evaluating"):
        L = L.to(device)
        ab_real = ab_real.to(device)
        
        ab_fake = generator(L)
        batch_metrics = evaluator.evaluate_batch(L, ab_fake, ab_real)
        
        for key in metrics_sum:
            metrics_sum[key] += batch_metrics[key]
        
        num_evaluated += 1

# Average metrics
metrics_avg = {key: val / num_evaluated for key, val in metrics_sum.items()}

print(f"\n{'='*60}")
print(f"Model {MODEL_ID} Evaluation Results:")
print(f"{'='*60}")
print(f"PSNR:        {metrics_avg['PSNR']:.4f} dB")
print(f"SSIM:        {metrics_avg['SSIM']:.4f}")
print(f"CIEDE2000:   {metrics_avg['CIEDE2000']:.4f}")
print(f"LPIPS:       {metrics_avg['LPIPS']:.4f}")
print(f"{'='*60}\n")

## Save Results

In [ ]:
# Save metrics to CSV
metrics_df = pd.DataFrame([{
    'Model_ID': MODEL_ID,
    'Architecture': 'U-Net',
    'GAN': USE_GAN,
    'L1_Loss': USE_L1,
    'Perceptual_Loss': USE_PERCEPTUAL,
    'Lambda_L1': LAMBDA_L1,
    'Lambda_Perceptual': LAMBDA_PERCEPTUAL,
    'PSNR': metrics_avg['PSNR'],
    'SSIM': metrics_avg['SSIM'],
    'CIEDE2000': metrics_avg['CIEDE2000'],
    'LPIPS': metrics_avg['LPIPS'],
    'Best_Val_Loss': best_val_loss,
    'Epochs': EPOCHS
}])

csv_path = results_dir / f'metrics_{MODEL_ID}.csv'
metrics_df.to_csv(csv_path, index=False)
print(f"✓ Metrics saved to: {csv_path}")

# Save training history plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Generator loss with train and validation
ax1.plot(history['epoch'], history['gen_loss'], 'b-', linewidth=2, label='Generator Loss (Train)', marker='o', markersize=4)
ax1.plot(history['epoch'], history['val_loss'], 'r-', linewidth=2, label='Validation Loss', marker='s', markersize=4)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Generator Training History', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)

# Discriminator loss
ax2.plot(history['epoch'], history['disc_loss'], 'g-', linewidth=2, label='Discriminator Loss', marker='^', markersize=4)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.set_title('Discriminator Training History', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11)

plt.suptitle(f'Model {MODEL_ID} Training History', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()

plot_path = results_dir / f'training_history_{MODEL_ID}.png'
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f"✓ Training plot saved to: {plot_path}")
plt.show()

In [ ]:
# Visualize and save sample results
generator.eval()
num_samples = 5

with torch.no_grad():
    L, ab_real, gray_img = next(iter(test_loader))
    L = L[:num_samples].to(device)
    ab_real = ab_real[:num_samples]
    gray_img = gray_img[:num_samples]
    
    ab_fake = generator(L).cpu()
    L = L.cpu()
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4 * num_samples))
    
    for i in range(num_samples):
        # Grayscale input
        axes[i, 0].imshow(gray_img[i].squeeze(), cmap='gray')
        axes[i, 0].set_title('Input (Grayscale)', fontweight='bold')
        axes[i, 0].axis('off')
        
        # Generated colorization
        L_np = ((L[i].squeeze().numpy() + 1.0) * 50.0)
        ab_fake_np = ab_fake[i].permute(1, 2, 0).numpy() * 128.0
        lab_fake = np.zeros((IMG_SIZE, IMG_SIZE, 3))
        lab_fake[:, :, 0] = L_np
        lab_fake[:, :, 1:] = ab_fake_np
        rgb_fake = lab2rgb(lab_fake)
        
        axes[i, 1].imshow(rgb_fake)
        axes[i, 1].set_title('Generated Color', fontweight='bold')
        axes[i, 1].axis('off')
        
        # Ground truth
        ab_real_np = ab_real[i].permute(1, 2, 0).numpy() * 128.0
        lab_real = np.zeros((IMG_SIZE, IMG_SIZE, 3))
        lab_real[:, :, 0] = L_np
        lab_real[:, :, 1:] = ab_real_np
        rgb_real = lab2rgb(lab_real)
        
        axes[i, 2].imshow(rgb_real)
        axes[i, 2].set_title('Ground Truth', fontweight='bold')
        axes[i, 2].axis('off')
    
    plt.suptitle(f'Model {MODEL_ID} - Colorization Results', fontsize=16, fontweight='bold', y=1.0)
    plt.tight_layout()
    
    output_path = results_dir / f'colorization_results_{MODEL_ID}.png'
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    print(f"✓ Visualization saved to: {output_path}")
    
    plt.show()

print(f"\n{'='*60}")
print(f"Model {MODEL_ID} - Training and Evaluation Complete!")
print(f"{'='*60}")
print(f"Models saved in: {save_dir}")
print(f"Results saved in: {results_dir}")
print(f"{'='*60}")